In [58]:
from Bio import Entrez
from Bio.Entrez import esearch, esummary, read, efetch
from pathos.multiprocessing import Pool
from lxml import etree as ET
import pandas as pd
import gc
import numpy as np
from p_tqdm import p_map
import string

**What we need**

*Pubmed*
- [x] publication date
- [x] PubmedID
- [x] Authors
- [x] Affiliations
- [x] Keywords
- [x] MeSH Descriptors
- [x] MeSH Qualifiers
- [x] MeSH hierarchy
- [x] Abstract available?
- [x] Abstract
- [ ] Free Full Text available? (not in esummary, check via PMC)
- [ ] Abstract/Fulltext Scraper for Editorials and stuffs

In [1006]:
%qtconsole

In [1236]:
class NCBITool(object):
    
  
    def __init__(self, db='pubmed'):
        self.db = db
        print('Setting Database: ', db)
        
        parsers = {
            'pubmed':NCBITool.parse_pubmed_ids,
            'pmc':NCBITool.parse_pmc_ids
        }
    
        self.parser = parsers[self.db]
        
    def search(self,
               search_term,
               retmax=2000):
        
        handle = esearch(db=self.db, term=search_term, retmax = retmax)
        
        self.search_result = read(handle)
        self.ids = [str(i) for i in self.search_result.get('IdList')]
        
        print('No. of results: ', len(self.ids))
        if len(self.ids)==retmax:
            print('###! There might be more results available !###')
            
    def parse(self, n_jobs=12, ids=None, keep_xml=False):
        
        if not hasattr(self, 'search_result') and ids==None:
            print('###! No ids Provided - Aborting !###')
            return
        
        elif ids==None:
            ids = self.ids
                        
        if n_jobs > len(self.ids):
            n_jobs = len(self.ids)
            
        with Pool(processes=n_jobs) as pool:
            data =pool.map(
                    self.parser,
                    list([list(chunk) for chunk in np.array_split(ids, n_jobs)]),
                )
            
#         data = p_map(self.parser,
#                      [list(chunk) for chunk in np.array_split(ids, n_jobs)],
#                      num_cpus=n_jobs)

        column_names = {
            'pubmed':[
                'pubmed_id',
               'title',
               'authors',
               'affiliations',
               'pub_date',
               'abstract',
               'doi',
               'pmcid',
               'journal',
               'pubmed_class',
               'pubmed_keywords',
               'mesh_descriptors',
               'mesh_qualifiers'
            ],
            'pmc':[
                'pmcid',
                'title',
                'authors',
                'affiliations',
                'pub_date',
                'pubmed_id',
                'doi',
                'abstract',
                'full_text',
                'journal',
                'pmcclass',
                'pmc_keywords'
            ]
        }

        self.data = pd.concat(
            [
                pd.DataFrame(
                    x[0],
                    columns = column_names[self.db]
                )
                for x in data
            ]
        )
        
        self.data.pub_date = pd.to_datetime(self.data.pub_date)
    
    @classmethod
    def parse_pmc_ids(self, pmcid, retmode = 'xml'):
        
        #get xml from pmc
        handle = efetch(db='pmc', id = pmcid, retmode = retmode)
        xml_string = handle.read()
        xml = ET.fromstring(xml_string)

        #check for keywords and MeshTerms
        keys = []
        
        for art in xml.getchildren():
            
            #title
            title = ''.join(art.xpath('.//article-meta//article-title/text()'))
            
            #authors
            auth = zip(art.findall('.//*[@contrib-type="author"]/name/given-names'),
                       art.findall('.//*[@contrib-type="author"]/name/surname'))
            auth = ';'.join([' '.join([i.text, j.text]) for (i,j) in auth])
            
            #affiliations
            aff = ';'.join(art.xpath('.//aff/text()'))
            
            #publication_date
            pub_date = '-'.join(art.xpath('.//article-meta/pub-date[@pub-type="epub"]/*/text()')[::-1])
            
            #ids
            ##pubmed_id
            pubmed_id = ''.join(art.xpath('.//article-meta/article-id[@pub-id-type="pmid"]/text()'))
            ##doi
            doi = ''.join(art.xpath('.//article-meta/article-id[@pub-id-type="doi"]/text()'))
            ##pmcid
            pmcid = ''.join(art.xpath('.//article-meta/article-id[@pub-id-type="pmc"]/text()'))
            
            #abstract
            abstract = ''.join(art.xpath('.//abstract//*/text()'))
            
            #fulltext
            full_text = ''.join(art.xpath('.//body//*/text()'))
            
            #journal
            journal = ''.join(art.xpath('//journal-meta/journal-id[@journal-id-type="iso-abbrev"]/text()'))
            
            #pmcclass
            pmcclass = ''.join(art.xpath('.//article-meta/article-categories//subject/text()'))

            #pmc_keywords
            pmc_keywords = art.xpath('.//kwd/text()')
        
            keys.append(
                (
                    pmcid,
                    title,
                    auth,
                    aff,
                    pub_date,
                    pubmed_id,
                    doi,
                    abstract,
                    full_text,
                    journal,
                    pmcclass,
                    pmc_keywords
                )
            )
            
        return keys, xml_string
        
    
    @classmethod
    def parse_pubmed_ids(cls, pub_id, retmode='xml'): 
    
        #get xml from pubmed
        handle = efetch(db='pubmed', id = pub_id, retmode = retmode)
        xml_string = handle.read()
        xml = ET.fromstring(xml_string)

        #check for keywords and MeshTerms
        keys = []

        for art in xml.getchildren():
            
            #authors
            auth = zip(art.findall('.//Author/ForeName'), art.findall('.//Author/LastName'))
            auth = ';'.join([' '.join([i.text, j.text]) for (i,j) in auth])
            
            #affiliations
            email_re = re.compile('Electronic address.*\.')
            aff = ';'.join(
                        np.unique(
                            [
                                email_re.sub('',aff.text).strip(string.punctuation+' ')
                                for aff in art.findall('.//Affiliation')
                            ]
                        )
                    )   
            
            #publication date
            pub_date = '-'.join(
                [
                    x.text for x in [
                        art.find('.//PubMedPubDate[@PubStatus="entrez"]/Year'),
                        art.find('.//PubMedPubDate[@PubStatus="entrez"]/Month'),
                        art.find('.//PubMedPubDate[@PubStatus="entrez"]/Day')
                    ]
                ]
            )
            
            if pub_date == '':
                pub_date = '1900-01-01'
                
            pubmed_id = ''.join(art.xpath('.//PubmedData/ArticleIdList/ArticleId[@IdType="pubmed"]/text()'))
            doi = ''.join(art.xpath('.//PubmedData/ArticleIdList/ArticleId[@IdType="doi"]/text()'))
            pmcid =  ''.join(art.xpath('.//PubmedData/ArticleIdList/ArticleId[@IdType="pmc"]/text()'))
            title = ''.join(art.xpath('.//ArticleTitle/text()')).strip()
            
            abstract = ''.join(art.xpath('.//AbstractText/text()'))
            journal = ''.join(art.xpath('.//Journal/ISOAbbreviation/text()'))
            pubmed_class = ','.join(art.xpath('.//PublicationType/text()'))
            keys.append((pubmed_id,
                         title,
                         auth,
                         aff,
                         pub_date,
                         abstract,
                         doi,
                         pmcid,
                         journal,
                         pubmed_class,
                         art.xpath('MedlineCitation/KeywordList/*/text()'),
                         art.xpath('MedlineCitation/MeshHeadingList/MeshHeading/*[@MajorTopicYN="Y"]/../DescriptorName/text()'),
                         art.xpath('MedlineCitation/MeshHeadingList/MeshHeading/*[@MajorTopicYN="Y"]/../QualifierName/text()')))


#         df = pd.DataFrame(keys, columns=['pubmed_id', 'pub_date', 'keywords', 'mesh_descriptors', 'mesh_qualifiers'])
#         df.pub_date = pd.to_datetime(df.pub_date)

        gc.collect()

        return keys, xml_string

In [1103]:
df, x = NCBITool.parse_pmc_ids('7062204')
art = ET.fromstring(x)

In [1056]:
Entrez.email = 'heiner.atze@crc.jussieu.fr'

In [1057]:
search_term = 'COVID-19 OR 2019-nCOV OR SARS-Cov-2 OR "novel coronavirus" AND (2019/11[DP] OR 2019/12[DP] OR 2020[DP])'

In [1331]:
p = NCBITool('pmc')

Setting Database:  pmc


In [1332]:
p.search(search_term)

No. of results:  187


In [1139]:
p.parse(4)

In [1142]:
p.data.loc[p.data.pubmed_id=='']

,pmcid,title,authors,affiliations,pub_date,pubmed_id,doi,abstract,full_text,journal,pmcclass,pmc_keywords
0,7075990,From SARS to COVID-19: A previously unknown SA...,Mohamed E. El Zowalaty;Josef D. Järhult,"Zoonosis Science Center, Department of Medical...",2020-02-24,,10.1016/j.onehlt.2020.100124,Human coronaviruses continue to pose a threat ...,"Today, the world faces many complex problems, ...",One HealthJ Clin MedJ Clin MedJ Clin MedJ Clin...,Short Communication,[]
11,7074995,SARS-CoV-2 and COVID-19: The most important re...,Kit-San Yuen;Zi -Wei Ye;Sin-Yee Fung;Chi-Ping ...,"3/F Laboratory Block, 21 Sassoon Road, Pokfula...",2020-03-16,,10.1186/s13578-020-00404-4,Coronavirus disease 2019 (COVID-19) caused by ...,The 2019-nCoV causes an ongoing outbreak of lo...,One HealthJ Clin MedJ Clin MedJ Clin MedJ Clin...,Research Highlight,"[SARS-CoV-2, COVID-19, 2019 novel coronavirus ..."
14,7062204,Prediction of the SARS-CoV-2 (2019-nCoV) 3C-li...,Yu Wai Chen;Chin-Pang Bennu Yiu;Kwok-Yin Wong;...,\n;Department of Applied Biology & Chemical Te...,2020-02-21,,10.12688/f1000research.22457.1,We prepared the three-dimensional model of the...,"\n Introduction\n On 7 January 2020,...",One HealthJ Clin MedJ Clin MedJ Clin MedJ Clin...,Brief ReportArticles,"[COVID-19, SARS, 2019-nCoV, 3C-like protease, ..."
16,7075270,Information Typology in Coronavirus (COVID-19)...,Hasan Ashrafi-rizi;Zahra Kazempour,\n;Medical Library and Information Science Dep...,2020-03-12,,,,\n Introduction:\n In late 2019 and ...,One HealthJ Clin MedJ Clin MedJ Clin MedJ Clin...,Letter to Editor,"[Coronavirus, Disaster Planning, Confidentiali..."
30,7060195,The outbreak of SARS-CoV-2 pneumonia calls for...,Weilong Shang;Yi Yang;Yifan Rao;Xiancai Rao,"400038 Chongqing, China",2020-03-06,,10.1038/s41541-020-0170-0,The outbreak of 2019-novel coronavirus disease...,An outbreak of 2019-novel coronavirus (SARS-Co...,One HealthJ Clin MedJ Clin MedJ Clin MedJ Clin...,Comment,"[Vaccines, Microbiology, Pathogenesis, Infecti..."
32,7071719,Dietary restriction of amino acids for Cancer ...,Jian-Sheng Kang,"Zhengzhou, 450052 China",2020-03-14,,10.1186/s12986-020-00439-x,"Biosyntheses of proteins, nucleotides and fatt...",\n Introduction\n Cancer is a comple...,One HealthJ Clin MedJ Clin MedJ Clin MedJ Clin...,Review,"[Amino acid restriction, Cancer, Lysine, Kwash..."
35,7068636,Transmission potential of the novel coronaviru...,Kenji Mizumoto;Gerardo Chowell,Graduate School of Advanced Integrated Studies...,2020-02-29,,10.1016/j.idm.2020.02.003,An outbreak of COVID-19 developed aboard the P...,\n Introduction\n While the novel co...,One HealthJ Clin MedJ Clin MedJ Clin MedJ Clin...,Special issue on Modelling and Forecasting the...,"[Corona, Epidemic, Confined settings, Cruise, ..."
39,7063273,CT imaging of two cases of one family cluster ...,Xiaofei Hu;Jiafei Chen;Xiaomei Jiang;Shiqi Tao...,"Department of Radiology, Southwest Hospital, ;...",NaT,,10.21037/qims.2020.02.10,,,One HealthJ Clin MedJ Clin MedJ Clin MedJ Clin...,Letter to the Editor,[]
41,7075272,A Novel Coronavirus Outbreak from Wuhan City i...,Mostafa Alavi-Moghaddam,"\n;Emergency Medicine Department, Imam Hossein...",2020-02-02,,,,"Dear Editor On 31 December 2019, Chinese autho...",One HealthJ Clin MedJ Clin MedJ Clin MedJ Clin...,Letter to Editor,[]
6,7064018,"Coronavirus infections: Epidemiological, clini...",Didier Raoult;Alimuddin Zumla;Franco Locatelli...,"\n;Aix-Marseille Univ., Institut de Recherche ...",2020-03-02,,10.15698/cst2020.04.216,Coronaviruses (CoVs) are a large family of env...,\n INTRODUCTION\n Infections have be...,Cardiovasc DiabetolGlobal HealthTrop Med Healt...,EditorialCoronavirusMERS-CoVSARS-CoVsARS-CoV-2...,"[Coronavirus, MERS-CoV, SARS-CoV, sARS-CoV-2, ..."


In [1329]:
s = NCBITool()

Setting Database:  pubmed


In [1330]:
s.search(search_term)

No. of results:  1475


In [1101]:
s.data.head()

,pubmed_id,title,authors,affiliations,pub_date,abstract,doi,pmcid,journal,pubmed_class,keywords,mesh_descriptors,mesh_qualifiers
0,32199182,The emotional impact of Coronavirus 2019-nCoV ...,Carlos Kennedy Tavares Lima;Poliana Moreira de...,"PostGraduate Program in Health Sciences, Schoo...",2020-03-22,A novel form of Coronavirus (2019-nCoV) in Wuh...,https://www.doi.org/10.1016/j.psychres.2020.11...,,Psychiatry Res,Letter,"[Coronavirus (2019-nCoV), Emergencies, Mental ...",[],[]
1,32199142,Asymptomatic SARS-CoV-2 infected patients with...,Zhoukun Ling;Xi Xu;Qingxin Gan;Lieguang Zhang;...,"Department of Radiology, Guangzhou Eighth Peop...",2020-03-22,,https://www.doi.org/10.1016/j.ejrad.2020.108956,,Eur J Radiol,Letter,[],[],[]
2,32199105,Deciphering the power of isolation in controll...,Yan Niud;Fujie Xu,Collaborative Innovation Center for Diagnosis ...,2020-03-22,,https://www.doi.org/10.1016/S2214-109X(20)30085-1,,Lancet Glob Health,Journal Article,[],[],[]
3,32199083,The resilience of the Spanish health system ag...,Helena Legido-Quigley;José Tomás Mateos-García...,"Department of Global Health and Development, L...",2020-03-22,,https://www.doi.org/10.1016/S2468-2667(20)30060-8,,Lancet Public Health,Journal Article,[],[],[]
4,32199076,COVID-19 cacophony: is there any orchestra con...,Antoine Flahault,"Institute of Global Health, Faculty of Medicin...",2020-03-22,,https://www.doi.org/10.1016/S0140-6736(20)30675-9,,Lancet,Letter,[],[],[]


In [783]:
s.parse(8)

In [784]:
s.data.pub_date = pd.to_datetime(s.data.pub_date)

In [785]:
s.data.doi = s.data.doi.apply(lambda x: 'https://www.doi.org/'+x)

In [786]:
ind = s.data.abstract == ''

In [787]:
ind.shape

(1447,)

In [934]:
s.data.columns

Index(['pubmed_id', 'title', 'authors', 'affiliations', 'pub_date', 'abstract',
       'doi', 'pmcid', 'journal', 'pubmed_class', 'keywords',
       'mesh_descriptors', 'mesh_qualifiers'],
      dtype='object')

In [797]:
s.data.pub_date[~(s.data.pmcid == '')]

98    2020-03-21
99    2020-03-21
100   2020-03-21
101   2020-03-21
102   2020-03-21
         ...    
160   2020-01-26
170   2020-01-16
172   2019-12-21
173   2019-12-13
178   2019-11-07
Name: pub_date, Length: 159, dtype: datetime64[ns]

In [788]:
na = s.data.loc[ind]

In [789]:
na.pubmed_class.value_counts().head(10)

Journal Article                 325
Letter                          182
Editorial                       104
Journal Article,Comment          15
Letter,Comment                   13
News                             12
Editorial,Comment                 6
Case Reports,Journal Article      6
Journal Article,Review            5
Case Reports,Letter               4
Name: pubmed_class, dtype: int64

In [790]:
na.journal.value_counts().head(10)

BMJ                      75
Lancet                   59
JAMA                     28
N. Engl. J. Med.         26
Lancet Infect Dis        23
Travel Med Infect Dis    22
J. Hosp. Infect.         16
J. Infect.               14
Radiology                14
Intensive Care Med       13
Name: journal, dtype: int64

In [791]:
pd.crosstab(na.pubmed_class, na.journal).sort_values('BMJ', ascending=False)[['BMJ', 'Lancet', 'JAMA',
                                                             'N. Engl. J. Med.']].head(10)

journal,BMJ,Lancet,JAMA,N. Engl. J. Med.
pubmed_class,,,,
Journal Article,61,19,28,9
Editorial,9,1,0,5
Letter,2,25,0,6
"Journal Article,Published Erratum",1,0,0,0
"Editorial,Comment",1,2,0,0
"Letter,Comment",1,4,0,0
"Journal Article,Retraction of Publication",0,0,0,0
"News,Research Support, Non-U.S. Gov't",0,0,0,0
News,0,1,0,0


**BMJ**

- News (__Journal Article__ in PubMed)
- Editorial (as such in PubMed)

**JAMA**

- Viewpoint
- Research Letter (__Journal Article__ in PubMed)

**Lancet**

- Correspondence (listed as __Letters__ in PubMEd)
- Comment (__Journal Article__ in PubMed)

**NEJM**

- Correspondence (listed as __Letters__ in PubMEd)

In [793]:
na.loc[na.journal == 'N. Engl. J. Med.']#N. Engl. J. Med.

,pubmed_id,title,authors,affiliations,pub_date,abstract,doi,pmcid,journal,pubmed_class,keywords,mesh_descriptors,mesh_qualifiers
63,32197002,Covid-19 and the Stiff Upper Lip - The Pandemi...,David J Hunter,From the Nuffield Department of Population Hea...,2020-03-21,,https://www.doi.org/10.1056/NEJMp2005755,,N. Engl. J. Med.,Journal Article,[],[],[]
16,32187489,Audio Interview: New Research on Possible Trea...,Eric J Rubin;Lindsey R Baden;Stephen Morrissey,,2020-03-19,,https://www.doi.org/10.1056/NEJMe2005759,,N. Engl. J. Med.,Editorial,[],[],[]
18,32187463,Covid-19 - The Search for Effective Therapy.,Lindsey R Baden;Eric J Rubin,,2020-03-19,,https://www.doi.org/10.1056/NEJMe2005477,,N. Engl. J. Med.,Editorial,[],[],[]
19,32187460,Covid-19 - The Law and Limits of Quarantine.,Wendy E Parmet;Michael S Sinha,"From the Center for Health Policy and Law, Nor...",2020-03-19,,https://www.doi.org/10.1056/NEJMp2004211,,N. Engl. J. Med.,Journal Article,[],[],[]
20,32187459,"Facing Covid-19 in Italy - Ethics, Logistics, ...",Lisa Rosenbaum,Dr. Rosenbaum is a national correspondent for ...,2020-03-19,,https://www.doi.org/10.1056/NEJMp2005492,,N. Engl. J. Med.,Journal Article,[],[],[]
21,32187458,SARS-CoV-2 Infection in Children.,Xiaoxia Lu;Liqiong Zhang;Hui Du;Jingjing Zhang...,"Beijing Children's Hospital, Beijing, China;Ch...",2020-03-19,,https://www.doi.org/10.1056/NEJMc2005073,,N. Engl. J. Med.,Letter,[],[],[]
72,32182409,Aerosol and Surface Stability of SARS-CoV-2 as...,Neeltje van Doremalen;Trenton Bushmaker;Dylan ...,"Centers for Disease Control and Prevention, At...",2020-03-18,,https://www.doi.org/10.1056/NEJMc2004973,,N. Engl. J. Med.,Letter,[],[],[]
124,32163699,History in a Crisis - Lessons for Covid-19.,David S Jones,From the Department of Global Health and Socia...,2020-03-13,,https://www.doi.org/10.1056/NEJMp2004361,,N. Engl. J. Med.,Journal Article,[],[],[]
125,32163698,SARS-CoV-2 Infection among Travelers Returning...,Oon-Tek Ng;Kalisvar Marimuthu;Po-Ying Chia;Van...,"KK Women's and Children's Hospital, Singapore,...",2020-03-13,,https://www.doi.org/10.1056/NEJMc2003100,,N. Engl. J. Med.,Letter,[],[],[]
126,32163697,Detection of Covid-19 in Children in Early Jan...,Weiyong Liu;Qi Zhang;Junbo Chen;Rong Xiang;Hui...,"Central South University, Changsha, China;Fuda...",2020-03-13,,https://www.doi.org/10.1056/NEJMc2003717,,N. Engl. J. Med.,Letter,[],[],[]


In [777]:
df, x = NCBITool.parse_pubmed_ids('32083643')

In [778]:
art = ET.fromstring(x)

In [779]:
art.xpath('.//PublicationType/text()')

['Journal Article']

In [682]:
meta = pd.read_csv('../all_sources_metadata_2020-03-13.csv')

In [806]:
s.data.pubmed_id.dtype

dtype('O')

In [938]:
s.data.pmcid.unique()

array(['', 'PMC7075990', 'PMC7060195', 'PMC7078228', 'PMC7073332',
       'PMC7062204', 'PMC7068636', 'PMC7063273', 'PMC7074995',
       'PMC7075675', 'PMC7075270', 'PMC7075271', 'PMC7075272',
       'PMC7077011', 'PMC7076992', 'PMC7081176', 'PMC7081171',
       'PMC7048972', 'PMC7049028', 'PMC7050214', 'PMC7050235',
       'PMC7050215', 'PMC7073313', 'PMC7068984', 'PMC7070509',
       'PMC7061470', 'PMC7061485', 'PMC7067954', 'PMC7069035',
       'PMC7081173', 'PMC7065369', 'PMC7081175', 'PMC7081177',
       'PMC7066521', 'PMC7050133', 'PMC7068161', 'PMC7068166',
       'PMC7068163', 'PMC7068162', 'PMC7068164', 'PMC7057189',
       'PMC7036635', 'PMC7036629', 'PMC7036645', 'PMC7036705',
       'PMC7062433', 'PMC7063757', 'PMC7081172', 'PMC7052559',
       'PMC7045882', 'PMC7045878', 'PMC7045880', 'PMC7045877',
       'PMC7049657', 'PMC7081812', 'PMC7035340', 'PMC7040011',
       'PMC7039910', 'PMC7047037', 'PMC7047033', 'PMC7075079',
       'PMC7057172', 'PMC7054527', 'PMC7055041', 'P

In [814]:
meta_ind = np.isin(s.data.pubmed_id[~ind].replace('', np.nan).dropna().values.astype(float),meta.pubmed_id.values)

In [936]:
s.data[['pub_date', 'pubmed_id', 'abstract']].loc[~ind].replace('', np.nan).dropna().loc[~meta_ind]

,pub_date,pubmed_id,abstract
0,2020-03-22,32199182,A novel form of Coronavirus (2019-nCoV) in Wuh...
5,2020-03-22,32199075,"Public health measures to prevent, detect, and..."
9,2020-03-22,32198989,新型冠状病毒肺炎疫情期间法医现场勘验与尸体检验防护.摘要：.法医病理学；尸体检验；新型冠状病...
10,2020-03-22,32198988,新型冠状病毒疫情期法医学尸体检验的安全防护.摘要：.法医病理学；尸体解剖；冠状病毒；新型冠状...
11,2020-03-22,32198987,新型冠状病毒肺炎死亡尸体系统解剖大体观察报告.摘要：.法医病理学；尸体解剖；冠状病毒；新型冠...
...,...,...,...
174,2019-12-10,31813836,Our understanding of climate factors and their...
175,2019-11-16,31727466,Since the first case of human infection by the...
176,2019-11-13,31712093,Porcine epidemic diarrhea virus (PEDV) is a hi...
177,2019-11-07,31690127,Coronaviruses (CoVs) infect humans and multipl...


In [768]:
(meta.pubmed_id == '32050060').sum()

/home/kantundpeterpan/anaconda3/envs/bio/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


0

In [769]:
from io import StringIO

In [811]:
s.data.pubmed_id[~ind].replace('', np.nan).dropna().values.astype(float).shape

(753,)

In [812]:
ind.sum()

693